#Building Multi-Agent Systems: Agent-to-Agent Communication

## A call_agent Tool

Imagine we want to create a system where multiple specialized agents can work together, each bringing their unique capabilities to solve complex problems. For example, we might have a primary agent that coordinates high-level tasks but needs to delegate specialized work to other agents. To make this possible, we need a way for agents to communicate with each other.

As we have seen in the past, often the most effective way to add a capability to an agent is to expose it as a tool. This architectural approach makes it easy to create systems with multi-agent coordination by simply exposing the right tool interfaces between them. Let’s build a multi-agent collaboration capability by creating a `call_agent` tool that allows one agent to invoke another and receive its results. We’ll see how the ActionContext makes this surprisingly straightforward.

First, let’s examine how the tool will work within an agent’s execution:

In [ ]:
@register_tool()
def call_agent(action_context: ActionContext,
               agent_name: str,
               task: str) -> dict:
    """
    Invoke another agent to perform a specific task.

    Args:
        action_context: Contains registry of available agents
        agent_name: Name of the agent to call
        task: The task to ask the agent to perform

    Returns:
        The result from the invoked agent's final memory
    """
    # Get the agent registry from our context
    agent_registry = action_context.get_agent_registry()
    if not agent_registry:
        raise ValueError("No agent registry found in context")

    # Get the agent's run function from the registry
    agent_run = agent_registry.get_agent(agent_name)
    if not agent_run:
        raise ValueError(f"Agent '{agent_name}' not found in registry")

    # Create a new memory instance for the invoked agent
    invoked_memory = Memory()

    try:
        # Run the agent with the provided task
        result_memory = agent_run(
            user_input=task,
            memory=invoked_memory,
            # Pass through any needed context properties
            action_context_props={
                'auth_token': action_context.get('auth_token'),
                'user_config': action_context.get('user_config'),
                # Don't pass agent_registry to prevent infinite recursion
            }
        )

        # Get the last memory item as the result
        if result_memory.items:
            last_memory = result_memory.items[-1]
            return {
                "success": True,
                "agent": agent_name,
                "result": last_memory.get("content", "No result content")
            }
        else:
            return {
                "success": False,
                "error": "Agent failed to run."
            }

    except Exception as e:
        return {
            "success": False,
            "error": str(e)
        }

## Building a Meeting Scheduling System with Specialized Agents

First, let’s look at the tools available to our scheduling specialist. This agent needs to interact with calendars and create invites:

In [ ]:
@register_tool()
def check_availability(
    action_context: ActionContext,
    attendees: List[str],
    start_date: str,
    end_date: str,
    duration_minutes: int,
    _calendar_api_key: str
) -> List[Dict]:
    """Find available time slots for all attendees."""
    return calendar_service.find_available_slots(...)

@register_tool()
def create_calendar_invite(
    action_context: ActionContext,
    title: str,
    description: str,
    start_time: str,
    duration_minutes: int,
    attendees: List[str],
    _calendar_api_key: str
) -> Dict:
    """Create and send a calendar invitation."""
    return calendar_service.create_event(...)

# The scheduling specialist is focused entirely on finding times and creating meetings:

scheduler_agent = Agent(
    goals=[
        Goal(
            name="schedule_meetings",
            description="""Schedule meetings efficiently by:
            1. Finding times that work for all attendees
            2. Creating and sending calendar invites
            3. Handling any scheduling conflicts"""
        )
    ],
...
)

# Now let’s look at our project management agent. This agent focuses on project status and deciding when meetings are needed:

@register_tool()
def get_project_status(
    action_context: ActionContext,
    project_id: str,
    _project_api_token: str
) -> Dict:
    """Retrieve current project status information."""
    return project_service.get_status(...)

@register_tool()
def update_project_log(
    action_context: ActionContext,
    entry_type: str,
    description: str,
    _project_api_token: str
) -> Dict:
    """Record an update in the project log."""
    return project_service.log_update(...)

@register_tool()
def call_agent(
    action_context: ActionContext,
    agent_name: str,
    task: str
) -> Dict:
    """Delegate to a specialist agent."""
    # Implementation as shown in previous tutorial

# The project management agent uses these tools to monitor progress and arrange meetings when needed:

project_manager = Agent(
    goals=[
        Goal(
            name="project_oversight",
            description="""Manage project progress by:
            1. Getting the current project status
            2. Identifying when meetings are needed if there are issues in the project status log
            3. Delegating meeting scheduling to the "scheduler_agent" to arrange the meeting
            4. Recording project updates and decisions"""
        )
    ],
    ...
)

This division of responsibilities keeps each agent focused on its core competency:

- The project manager understands project status and when meetings are needed
- The scheduler excels at finding available times and managing calendar logistics
- The call_agent tool allows seamless collaboration between them

## The call_agent Tool
The call_agent tool manages several important aspects of agent interaction:

- Memory Isolation: Each invoked agent gets its own memory instance, preventing confusion between different agents’ conversation histories.

- Context Management: We carefully control what context properties are passed to the invoked agent, preventing infinite recursion while ensuring necessary resources are available.

- Result Handling: The tool extracts the final memory item as the result, providing a clean way to return information to the calling agent.

## Registering Agents
To make this system work, we need to register our agents in the registry:

In [ ]:
class AgentRegistry:
    def __init__(self):
        self.agents = {}

    def register_agent(self, name: str, run_function: callable):
        """Register an agent's run function."""
        self.agents[name] = run_function

    def get_agent(self, name: str) -> callable:
        """Get an agent's run function by name."""
        return self.agents.get(name)

# When setting up the system
registry = AgentRegistry()
registry.register_agent("scheduler_agent", scheduler_agent.run)

# Include registry in action context
action_context = ActionContext({
    'agent_registry': registry,
    # Other shared resources...
})

The ActionContext provides a clean way to make the agent registry available to the `call_agent` tool without exposing it directly to all tools. When an agent needs to delegate a task, it simply uses the tool like any other, and the environment system handles the details of finding and invoking the right agent.

This architecture allows us to build complex multi-agent systems where each agent maintains its specialization while being able to collaborate with other agents when needed. The memory isolation ensures that each agent works with a clean context, while the result extraction provides a standard way to pass information back to the calling agent.

# Memory Interaction Patterns in Multi-Agent Systems

When agents work together, how they share and manage memory dramatically affects their collaboration. Let’s explore different patterns for memory interaction between agents, understanding when each pattern is most useful and how to implement it.

## Message Passing: The Basic Pattern

The simplest form of agent interaction is message passing, where one agent sends a request and receives a response. This is like sending an email to a colleague - they get your message, do some work, and send back their results. You don’t see how they arrived at their answer; you just get their final response.

Here’s how we implement basic message passing:


In [ ]:
@register_tool()
def call_agent(action_context: ActionContext,
               agent_name: str,
               task: str) -> dict:
    """Basic message passing between agents."""
    agent_registry = action_context.get_agent_registry()
    agent_run = agent_registry.get_agent(agent_name)

    # Create fresh memory for the invoked agent
    invoked_memory = Memory()

    # Run agent and get result
    result_memory = agent_run(
        user_input=task,
        memory=invoked_memory
    )

    # Return only the final memory item
    return {
        "result": result_memory.items[-1].get("content", "No result")
    }

This pattern works well when the first agent only needs the final answer, not the reasoning process. For example, if a project manager agent asks a scheduling agent to find a meeting time, it might only need to know when the meeting was scheduled, not how the time was chosen.



##Memory Reflection: Learning from the Process
Sometimes we want the first agent to understand how the second agent reached its conclusion. This is like asking a colleague to not just give you their answer, but to explain their thought process. We can achieve this by copying all of the second agent’s memories back to the first agent:

In [ ]:
@register_tool()
def call_agent_with_reflection(action_context: ActionContext,
                             agent_name: str,
                             task: str) -> dict:
    """Call agent and receive their full thought process."""
    agent_registry = action_context.get_agent_registry()
    agent_run = agent_registry.get_agent(agent_name)

    # Create fresh memory for invoked agent
    invoked_memory = Memory()

    # Run agent
    result_memory = agent_run(
        user_input=task,
        memory=invoked_memory
    )

    # Get the caller's memory
    caller_memory = action_context.get_memory()

    # Add all memories from invoked agent to caller
    # although we could leave off the last memory to
    # avoid duplication
    for memory_item in result_memory.items:
        caller_memory.add_memory({
            "type": f"{agent_name}_thought",  # Mark source of memory
            "content": memory_item["content"]
        })

    return {
        "result": result_memory.items[-1].get("content", "No result"),
        "memories_added": len(result_memory.items)
    }

This pattern is valuable when the first agent needs to understand the reasoning process. For instance, if a research coordinator agent asks a data analysis agent to study some results, seeing the analysis process helps the coordinator better understand and use the conclusions.

## Memory Handoff: Continuing the Conversation
Sometimes we want the second agent to pick up where the first agent left off, with full context of what’s happened so far. This is like having a colleague step in to take over a project - they need to know everything that’s happened up to that point:

In [ ]:
@register_tool()
def hand_off_to_agent(action_context: ActionContext,
                      agent_name: str,
                      task: str) -> dict:
    """Transfer control to another agent with shared memory."""
    agent_registry = action_context.get_agent_registry()
    agent_run = agent_registry.get_agent(agent_name)

    # Get the current memory to hand off
    current_memory = action_context.get_memory()

    # Run agent with existing memory
    result_memory = agent_run(
        user_input=task,
        memory=current_memory  # Pass the existing memory
    )

    return {
        "result": result_memory.items[-1].get("content", "No result"),
        "memory_id": id(result_memory)
    }

This pattern is useful for complex tasks where context is crucial. For example, if a customer service agent hands off to a technical support agent, the technical agent needs to know the full history of the customer’s issue.

## Selective Memory Sharing: Using LLM Understanding for Context Selection

Sometimes we want an agent to intelligently choose which parts of its memory to share with another agent. Instead of using rigid rules, we can leverage the LLM’s understanding of context to select the most relevant memories for the task at hand.

Let’s implement a version of memory sharing that uses the LLM to analyze and select relevant memories with self-prompting:

In [ ]:
@register_tool(description="Delegate a task to another agent with selected context")
def call_agent_with_selected_context(action_context: ActionContext,
                                   agent_name: str,
                                   task: str) -> dict:
    """Call agent with LLM-selected relevant memories."""
    agent_registry = action_context.get_agent_registry()
    agent_run = agent_registry.get_agent(agent_name)

    # Get current memory and add IDs
    current_memory = action_context.get_memory()
    memory_with_ids = []
    for idx, item in enumerate(current_memory.items):
        memory_with_ids.append({
            **item,
            "memory_id": f"mem_{idx}"
        })

    # Create schema for memory selection
    selection_schema = {
        "type": "object",
        "properties": {
            "selected_memories": {
                "type": "array",
                "items": {
                    "type": "string",
                    "description": "ID of a memory to include"
                }
            },
            "reasoning": {
                "type": "string",
                "description": "Explanation of why these memories were selected"
            }
        },
        "required": ["selected_memories", "reasoning"]
    }

    # Format memories for LLM review
    memory_text = "\n".join([
        f"Memory {m['memory_id']}: {m['content']}"
        for m in memory_with_ids
    ])

    # Ask LLM to select relevant memories
    selection_prompt = f"""Review these memories and select the ones relevant for this task:

Task: {task}

Available Memories:
{memory_text}

Select memories that provide important context or information for this specific task.
Explain your selection process."""

    # Self-prompting magic to find the most relevant memories
    selection = prompt_llm_for_json(
        action_context=action_context,
        schema=selection_schema,
        prompt=selection_prompt
    )

    # Create filtered memory from selection
    filtered_memory = Memory()
    selected_ids = set(selection["selected_memories"])
    for item in memory_with_ids:
        if item["memory_id"] in selected_ids:
            # Remove the temporary memory_id before adding
            item_copy = item.copy()
            del item_copy["memory_id"]
            filtered_memory.add_memory(item_copy)

    # Run the agent with selected memories
    result_memory = agent_run(
        user_input=task,
        memory=filtered_memory
    )

    # Add results and selection reasoning to original memory
    current_memory.add_memory({
        "type": "system",
        "content": f"Memory selection reasoning: {selection['reasoning']}"
    })

    for memory_item in result_memory.items:
        current_memory.add_memory(memory_item)

    return {
        "result": result_memory.items[-1].get("content", "No result"),
        "shared_memories": len(filtered_memory.items),
        "selection_reasoning": selection["reasoning"]
    }

This implementation makes memory selection more intelligent and transparent:

Each memory gets assigned a unique ID for reference.

The complete set of memories is presented to the LLM with their IDs.

The LLM analyzes the memories in the context of the specific task and selects the relevant ones using structured JSON output.

The LLM provides reasoning for its selection, which is preserved in the original agent’s memory.

For example, if a project management agent is delegating a budget review task, the interaction might look like this:

In [ ]:
# Example memory contents:
memories = [
    {"type": "user", "content": "We need to build a new reporting dashboard"},
    {"type": "assistant", "content": "Initial cost estimate: $50,000"},
    {"type": "user", "content": "That seems high"},
    {"type": "assistant", "content": "Breakdown: $20k development, $15k design..."},
    {"type": "system", "content": "Project deadline updated to Q3"},
    {"type": "user", "content": "Can we reduce the cost?"}
]

# LLM's selection might return:
{
    "selected_memories": ["mem_1", "mem_3", "mem_5"],
    "reasoning": "Selected memories containing cost information and the request for cost reduction, excluding project timeline and general discussion as they're not directly relevant to the budget review task."
}

The second agent then receives only the memories about cost estimates, breakdowns, and the request for reduction, giving it focused context for its budget review task without extraneous information about timelines or other project aspects.

This approach has several advantages over rule-based filtering:

* The selection process can understand context and implications, not just match patterns.

* The reasoning is preserved, helping track why certain information was or wasn’t shared.

* The selection can adapt to different types of tasks and contexts without changing the code.

* The original agent maintains a record of what information was shared and why.

This pattern is valuable when you want to provide specific context without overwhelming the second agent with irrelevant information. For example, if a project planning agent asks a budget specialist to review costs, it might share only the memories related to resource allocation and expenses, not the entire project history.

## Recap of the Four Memory Sharing Patterns
Each of these patterns serves a different purpose in agent collaboration:

* Message passing keeps interactions simple and focused
* Memory reflection helps agents learn from each other’s processes
* Memory handoff enables seamless continuation of complex tasks
* Selective memory sharing provides relevant context while reducing noise

The choice of pattern depends on your specific needs:

* How much context does the second agent need?
* Does the first agent need to understand the second agent’s process?
* Should the conversation history be preserved?
* Is there sensitive information that should be filtered?

By understanding these patterns, you can design agent interactions that effectively balance information sharing with task focus, leading to more efficient and capable multi-agent systems.